# Data processing -- stale experiments

In [1]:
using Tidier

In [ ]:
using DBnomics
natl_acc = rdb("Eurostat", "nama_10_gdp", mask="B1GQ")

In [ ]:
using HTTP
res = HTTP.get("https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nama_10_gdp?format=TSV&compressed=true")

In [ ]:
import Downloads
using CSV
natl_acc = CSV.read(Downloads.download("https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nama_10_gdp?format=TSV&compressed=true"), DataFrame)

In [ ]:
using CSV
natl_acc = CSV.read("datasets/estat_nama_10_gdp.tsv.gz", DataFrame; delim="\t")

In [ ]:
natl_acc = read_csv("datasets/estat_nama_10_gdp.tsv.gz")

In [ ]:
natl_acc = DataFrames.rename(natl_acc, "freq,unit,na_item,geo\\TIME_PERIOD" => "mixed")

In [ ]:
nat_acc = @chain natl_acc begin
    @separate(mixed, (freq,unit,na_item,geo), ",")
    @pivot_longer(-(freq:geo), names_to=year,values_to=value)
    @select(-freq)
    @mutate(value = as_float(value), year = as_integer(str_squish(year)))
end

In [ ]:
bg_nat_acc = @chain nat_acc begin
    @filter(geo == "BG", unit ==  "CLV10_MNAC", na_item in ("B1GQ", "P51G"))
    @select(na_item,year,value)
    @drop_missing()
end

In [ ]:
# names(nat_acc)
println(@distinct(nat_acc.na_item))